In [1]:
%load_ext autoreload
%autoreload 2
import sklearn
from sklearn.preprocessing import StandardScaler
import sklearn.svm as svm
from sklearn.neural_network import (MLPClassifier, MLPRegressor)
import numpy as np
from agents.knn import *
from settings import settings

In [2]:
from agents.double_model_agent import extract_state

speed_actions_labels = [
    {'accel':0, 'brake': 0},
    {'accel':1, 'brake': 0},
    {'accel':0, 'brake': 1}
]
state_keys = [
    'speedX',
    'speedY',
    'speedZ',
    'angle',
    'trackPos',
#     'distFromStart',
    'track',
    'wheelSpinVel'
]

test_size = 0.05

In [3]:
data = [
    x for x in parse_dataset('/home/kku/Documents/engineering-thesis/important_logs') 
    if not (x['speedX'] == 0 and x['accel'] == 0)
]
states = np.array(extract_state(data, state_keys, data_is_array=True))
steer_actions = np.array([x['steer'] for x in data])
speed_actons = np.array([
    speed_actions_labels.index({
        'accel': x['accel'],
        'brake': x['brake']
    }) 
    for x in data
])

In [4]:
X, Y = sklearn.utils.shuffle(states, steer_actions)
X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X, Y, test_size=test_size)

In [5]:
scaler = StandardScaler()  
scaler.fit(X_train)  
# regressor = svm.SVR()
regressor = MLPRegressor((400, 40), activation='tanh')

print(scaler.transform(X_train).shape)
regressor.fit(
    scaler.transform(X_train),
    Y_train
) 

(29011, 28)


MLPRegressor(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(400, 40), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [6]:
regressor.score(
    scaler.transform(X_test),
    Y_test
)

0.8738904203096368

In [7]:
X, Y = sklearn.utils.shuffle(
    np.hstack((scaler.transform(states), steer_actions[:, np.newaxis])),
    speed_actons
)
X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X, Y, test_size=test_size)

In [8]:
classifier = MLPClassifier((300,30), activation='tanh')
classifier.fit(
    X_train,
    Y_train
)

MLPClassifier(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(300, 30), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [9]:
print(classifier.score(
    X_test,
    Y_test
))
sklearn.metrics.confusion_matrix(Y_test, classifier.predict(X_test))

0.9842829076620825


array([[ 139,   11,    3],
       [   7, 1332,    1],
       [   2,    0,   32]])

In [10]:
def dump():
    path = 'saved_model'
    print(f'Dumping model to: {path}')
    np.savez(
        'saved_model/parameters.npz',
        **{
            'state_keys': state_keys,
            'speed_actions_labels': speed_actions_labels,
        }
    )
    joblib.dump(scaler, 'saved_model/scaler')
    joblib.dump(regressor, 'saved_model/steer_regressor')
    joblib.dump(classifier, 'saved_model/speed_classifier')
dump()
    

Dumping model to: saved_model
